In [107]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import glob
import seaborn as sns
import os
import datetime as dt
import xgboost as xgb
from sklearn.svm import OneClassSVM
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import pandas as pd
import catboost
import sklearn
from sklearn.model_selection import cross_val_score, KFold, learning_curve, cross_validate, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
import operator
import matplotlib.pyplot as plt
import xgboost as xgb
import os
import seaborn as sns
import datetime
from time import strftime, gmtime, time
from dateutil.relativedelta import relativedelta
from tqdm import tqdm
from sklearn.model_selection import train_test_split

tqdm.pandas()

In [108]:
sns.set_context("poster") 
plt.style.use('seaborn-poster')

In [159]:
!pip install xgboost
!pip install sklearn

  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1309 sha256=52227aab819a6e2e57d2ededbfc112c954553694388a1f05d080a0a79347ce97
  Stored in directory: /Users/sebastiaan/Library/Caches/pip/wheels/e4/7b/98/b6466d71b8d738a0c547008b9eb39bf8676d1ff6ca4b22af1c
Successfully built sklearn


In [160]:
def preprocessWeather(path_url):
    df = pd.read_csv(path_url)
    df.columns = df.columns.str.replace(' ', '')
    df[['FH', 'T', 'RH']] = df[['FH', 'T', 'RH']] / 10
    df['YYYYMMDD'] = pd.to_datetime(df['YYYYMMDD'], format='%Y%m%d')
    df['date'] = df['YYYYMMDD'] +  pd.to_timedelta(df['HH'], unit='h')
    df.drop(columns = ['#STN', 'DD', 'FF', 'FX', 'T10N', 'TD', 'Q', 
                       'P', 'VV', 'U', 'WW', 'IX', 'HH', 'YYYYMMDD'], inplace=True)
    df.set_index('date', inplace=True)
    return df

def preprocessResono(path_url):
    df = pd.read_csv(path_url)
    df = df.drop(columns = ["Unnamed: 0"])
    return df

In [161]:
df_Weather2020 = preprocessWeather("KNMI (Weather) 2020-2021/uurgeg_240_2011-2020.txt")
df_Weather2021 = preprocessWeather("KNMI (Weather) 2020-2021/uurgeg_240_2021-2030_new.txt")

df_Resono = preprocessResono("resono_2020_2022.csv")

/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_21339/2061503411.py:1: DtypeWarning: Columns (15,16,20,21,22,23,24) have mixed types.Specify dtype option on import or set low_memory=False.
  df_Weather2020 = preprocessWeather("KNMI (Weather) 2020-2021/uurgeg_240_2011-2020.txt")


In [162]:
df_weather = pd.concat([df_Weather2020, df_Weather2021], axis=0)
df_weather = df_weather.loc['2020-10':]

cols_int = ['SQ', 'DR', 'N', 'M', 'R', 'S', 'O', 'Y']
cols_float = ['FH', 'T']

df_weather[cols_float] = df_weather[cols_float].apply(pd.to_numeric, errors='coerce', axis=1)
df_weather[cols_int] = df_weather[cols_int].apply(pd.to_numeric, errors='coerce', axis=1)
df_weather['RH'] = df_weather['RH'].apply(lambda x: 0.05 if x==-0.1 else x)

In [163]:
df_weather_resample = pd.concat([df_weather[['FH', 'T', 'N']].resample('15T').interpolate(method='linear'),
                    df_weather[['RH', 'DR', 'SQ', 'M', 'R', 'S', 'O', 'Y']].resample('15T').bfill()],
                   axis=1)

df_weather_resample[['DR', 'SQ']] = df_weather_resample[['DR', 'SQ']] * 1.5
df_weather_resample['RH'] = df_weather_resample['RH'] / 4

In [164]:
#df_Resono = df_Resono_Oct_Dec.drop(columns = ["Unnamed: 0"])
df_Resono['End'] = pd.to_datetime(df_Resono['End'])
df_Resono['End'] = pd.to_datetime(df_Resono['End'].dt.strftime("%Y-%m-%d %H:%M:%S"))

In [165]:
df_Resono_index = df_Resono.set_index('End')
df_Resono_index = df_Resono_index.loc['2020-10':]

/var/folders/3y/wg1nvwfd2fgfgy_dxxxkcbsw0000gn/T/ipykernel_21339/325174172.py:2: FutureWarning: Value based partial slicing on non-monotonic DatetimeIndexes with non-existing keys is deprecated and will raise a KeyError in a future Version.
  df_Resono_index = df_Resono_index.loc['2020-10':]


In [166]:
merge_resono_weather = pd.merge(df_Resono_index, df_weather_resample, left_index=True, right_index=True, how='left')
merge_resono_weather = merge_resono_weather.rename({'T': 'Temperature', 'N': 'Clouds', 'FH': 'Windspeed',
                                                    'RH': 'Rain amount', 'DR': 'Rain duration' , 'SQ': 'Sun duration',
                                                    'M': 'Fog', 'R': 'Rain', 'S': 'Snow', 'O': 'Thunder', 'Y': 'Ice'},
                                                   axis=1) 
merge_resono_weather.head(5)

,Location,Visits,End_Dates,End_Time,Windspeed,Temperature,Clouds,Rain amount,Rain duration,Sun duration,Fog,Rain,Snow,Thunder,Ice
2020-10-01 00:15:00,Erasmuspark,243,2020-10-01,00:15:00,5.0,15.575,8.0,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Oosterpark,53,2020-10-01,00:15:00,5.0,15.575,8.0,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Rembrandtpark Noord,123,2020-10-01,00:15:00,5.0,15.575,8.0,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Rembrandtpark Zuid,138,2020-10-01,00:15:00,5.0,15.575,8.0,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Sarphatipark,96,2020-10-01,00:15:00,5.0,15.575,8.0,0.0,0.0,0.0,0,0,0,0,0


In [167]:
#Top 6 biggest parks
locations = ['Rembrandtpark Noord',
 'Rembrandtpark Zuid',
 'Sarphatipark',
 'Vondelpark Oost',
 'Vondelpark West',
 'Oosterpark',
 'Erasmuspark',
 'Westergasfabriek']

In [168]:
merge_resono_weather

,Location,Visits,End_Dates,End_Time,Windspeed,Temperature,Clouds,Rain amount,Rain duration,Sun duration,Fog,Rain,Snow,Thunder,Ice
2020-10-01 00:15:00,Erasmuspark,243,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Oosterpark,53,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Rembrandtpark Noord,123,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Rembrandtpark Zuid,138,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
2020-10-01 00:15:00,Sarphatipark,96,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-01-06 14:45:00,Sarphatipark,654,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
2022-01-06 14:45:00,Vondelpark West,332,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
2022-01-06 14:45:00,Westergasfabriek,68,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
2022-01-06 15:00:00,Sarphatipark,776,2022-01-06,15:00:00,5.0,5.200,4.00,0.0,0.0,15.0,0,0,0,0,0


In [169]:
def feature_importances(model) -> pd.DataFrame:

    importance_dict = {}

    for feature, importance in zip(X_train.columns, model.feature_importances_):
        importance_dict[feature] = importance

    df_importance = pd.DataFrame(sorted(importance_dict.items(), key=operator.itemgetter(1)))
    df_importance = df_importance.set_index(0).sort_values(by=[1], ascending=False)

    return df_importance

In [170]:
merge_resono_weather.reset_index(inplace=True)
merge_resono_weather

,index,Location,Visits,End_Dates,End_Time,Windspeed,Temperature,Clouds,Rain amount,Rain duration,Sun duration,Fog,Rain,Snow,Thunder,Ice
0,2020-10-01 00:15:00,Erasmuspark,243,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
1,2020-10-01 00:15:00,Oosterpark,53,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
2,2020-10-01 00:15:00,Rembrandtpark Noord,123,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
3,2020-10-01 00:15:00,Rembrandtpark Zuid,138,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
4,2020-10-01 00:15:00,Sarphatipark,96,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230993,2022-01-06 14:45:00,Sarphatipark,654,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
230994,2022-01-06 14:45:00,Vondelpark West,332,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
230995,2022-01-06 14:45:00,Westergasfabriek,68,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
230996,2022-01-06 15:00:00,Sarphatipark,776,2022-01-06,15:00:00,5.0,5.200,4.00,0.0,0.0,15.0,0,0,0,0,0


In [171]:
merge_resono_weather.drop(columns='index', inplace=True)
merge_resono_weather

,Location,Visits,End_Dates,End_Time,Windspeed,Temperature,Clouds,Rain amount,Rain duration,Sun duration,Fog,Rain,Snow,Thunder,Ice
0,Erasmuspark,243,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
1,Oosterpark,53,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
2,Rembrandtpark Noord,123,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
3,Rembrandtpark Zuid,138,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
4,Sarphatipark,96,2020-10-01,00:15:00,5.0,15.575,8.00,0.0,0.0,0.0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230993,Sarphatipark,654,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
230994,Vondelpark West,332,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
230995,Westergasfabriek,68,2022-01-06,14:45:00,5.0,5.400,3.25,0.0,0.0,15.0,0,0,0,0,0
230996,Sarphatipark,776,2022-01-06,15:00:00,5.0,5.200,4.00,0.0,0.0,15.0,0,0,0,0,0


In [210]:
X, y = merge_resono_weather.loc[:, merge_resono_weather.columns != "Visits"], merge_resono_weather[["Visits"]]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
cat_features = ['End_Dates','End_Time','Location']

X_train = X_train[['End_Dates','End_Time','Location','Windspeed','Temperature', 'Clouds', 'Rain amount', 'Rain duration','Sun duration', 'Fog','Rain','Snow','Thunder','Ice']]
X_test = X_test[['End_Dates','End_Time','Location','Windspeed','Temperature', 'Clouds', 'Rain amount', 'Rain duration','Sun duration', 'Fog','Rain','Snow','Thunder','Ice']]



assert "ARRIVAL_DELAY" not in X_train.columns
assert y_train.shape[1] == 1

In [217]:
cat_scores = {}

for iterations in range(25, 1000, 50):
    
    print(f"Fitting model for {iterations} iterations...")
    
    start_time = time()
    
    cat_scores[iterations] = {}
    
    cat_model = catboost.CatBoostRegressor(iterations=iterations, cat_features=cat_features, verbose=False)
    
    cat_model.fit(X_train, y_train, eval_set=(X_test, y_test))
    
    cat_scores[iterations]["train"] = cat_model.get_best_score()["learn"]["RMSE"]
    cat_scores[iterations]["test"] = cat_model.get_best_score()["validation"]["RMSE"]
    cat_scores[iterations]["time"] = time() - start_time
    
    print(f"Fitting model for {iterations} iterations took {time() - start_time} seconds...")

Fitting model for 25 iterations...
Fitting model for 25 iterations took 0.4600977897644043 seconds...
Fitting model for 75 iterations...
Fitting model for 75 iterations took 1.164438009262085 seconds...
Fitting model for 125 iterations...
Fitting model for 125 iterations took 1.668529987335205 seconds...
Fitting model for 175 iterations...
Fitting model for 175 iterations took 2.2835330963134766 seconds...
Fitting model for 225 iterations...
Fitting model for 225 iterations took 5.6830480098724365 seconds...
Fitting model for 275 iterations...
Fitting model for 275 iterations took 7.029798269271851 seconds...
Fitting model for 325 iterations...
Fitting model for 325 iterations took 8.427350044250488 seconds...
Fitting model for 375 iterations...
Fitting model for 375 iterations took 10.297098875045776 seconds...
Fitting model for 425 iterations...
Fitting model for 425 iterations took 11.248274803161621 seconds...
Fitting model for 475 iterations...
Fitting model for 475 iterations too

In [218]:
cat_scores

{25: {'train': 327.79283850771384,
  'test': 327.7791194109161,
  'time': 0.4600961208343506},
 75: {'train': 315.66081448827083,
  'test': 317.8207850705691,
  'time': 1.1644349098205566},
 125: {'train': 311.16559087633635,
  'test': 315.1889136050277,
  'time': 1.6685280799865723},
 175: {'train': 308.6010149834518,
  'test': 312.7424895414798,
  'time': 2.2835309505462646},
 225: {'train': 274.85711728995926,
  'test': 274.15474475367483,
  'time': 5.683045864105225},
 275: {'train': 270.84305712501066,
  'test': 271.02393386264345,
  'time': 7.029795169830322},
 325: {'train': 270.1629936307948,
  'test': 270.9796816041273,
  'time': 8.42734694480896},
 375: {'train': 267.97733158119473,
  'test': 268.7565530689443,
  'time': 10.297094106674194},
 425: {'train': 268.0644977518479,
  'test': 268.9018130663396,
  'time': 11.248271703720093},
 475: {'train': 265.3076819478044,
  'test': 266.10367890698456,
  'time': 12.66051983833313},
 525: {'train': 264.73748385618774,
  'test': 26

In [219]:
cat_model.get_best_score()

{'learn': {'RMSE': 258.0066590969808},
 'validation': {'RMSE': 262.10632055786357}}

In [220]:
feature_importances(cat_model)

,1
0,
End_Dates,36.532662
Location,19.806299
End_Time,19.276169
Temperature,11.084606
Windspeed,5.234684
Clouds,2.942675
Sun duration,1.941802
Rain amount,1.341412
Rain duration,1.186448


In [223]:
y_test

,Visits
164060,118
15295,10
53432,280
75727,4
17494,379
...,...
104269,19
98925,835
190066,73
47317,416
